In [1]:
import neo4j


# LLM and Embedding Model
from neo4j_graphrag.llm import OllamaLLM
from neo4j_graphrag.embeddings.ollama import OllamaEmbeddings


# Setup

In [2]:
neo4j_driver = neo4j.GraphDatabase.driver("bolt://neo4j:7687",
                auth=("neo4j", "password"))


In [8]:
llm=OllamaLLM(
   host="http://host.docker.internal:11434",
   model_name="qwen2.5",
   model_params={
       "response_format": {"type": "json_object"}, # use json_object formatting for best results
       "temperature": 0 # turning temperature down for more deterministic results
   }
)
#create text embedder
embedder = OllamaEmbeddings(model="nomic-embed-text", host="http://host.docker.internal:11434")

In [9]:
# Graph Schema Setup
basic_node_labels = ["Object", "Entity", "Group", "Person", "Organization", "Place"]

academic_node_labels = ["ArticleOrPaper", "PublicationOrJournal"]

medical_node_labels = ["Anatomy", "BiologicalProcess", "Cell", "CellularComponent",
                      "CellType", "Condition", "Disease", "Drug",
                      "EffectOrPhenotype", "Exposure", "GeneOrProtein", "Molecule",
                      "MolecularFunction", "Pathway"]

node_labels = basic_node_labels + academic_node_labels + medical_node_labels

# define relationship types
rel_types = ["ACTIVATES", "AFFECTS", "ASSESSES", "ASSOCIATED_WITH", "AUTHORED",
   "BIOMARKER_FOR"]

In [10]:


# define prompt template
prompt_template = '''
You are a medical researcher tasks with extracting information from papers
and structuring it in a property graph to inform further medical and research Q&A.

Extract the entities (nodes) and specify their type from the following Input text.
Also extract the relationships between these nodes. the relationship direction goes from the start node to the end node.


Return result as JSON using the following format:
{{"nodes": [ {{"id": "0", "label": "the type of entity", "properties": {{"name": "name of entity" }} }}],
  "relationships": [{{"type": "TYPE_OF_RELATIONSHIP", "start_node_id": "0", "end_node_id": "1", "properties": {{"details": "Description of the relationship"}} }}] }}

...

Use only fhe following nodes and relationships:
{schema}

Assign a unique ID (string) to each node, and reuse it to define relationships.
Do respect the source and target node types for relationship and the relationship direction.

Do not return any additional information other than the JSON in it.

Examples:
{examples}

Input text:

{text}
'''

# Build Graph

In [11]:
# Knowledge Graph Builder
from neo4j_graphrag.experimental.components.text_splitters.fixed_size_splitter import FixedSizeSplitter
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline

kg_builder_pdf = SimpleKGPipeline(
   llm=llm,
   driver=neo4j_driver,
   text_splitter=FixedSizeSplitter(chunk_size=500, chunk_overlap=100),
   embedder=embedder,
   entities=node_labels,
   relations=rel_types,
   prompt_template=prompt_template,
   from_pdf=True
)

pdf_file_paths = [
            # 'truncated-pdfs/biomolecules-11-00928-v2-trunc.pdf',
            # 'truncated-pdfs/GAP-between-patients-and-clinicians_2023_Best-Practice-trunc.pdf',
            'truncated-pdfs/pgpm-13-39-trunc.pdf'
]

for path in pdf_file_paths:
    print(f"Processing : {path}")
    pdf_result = await kg_builder_pdf.run_async(file_path=path)
    print(f"Result: {pdf_result}")

Processing : truncated-pdfs/pgpm-13-39-trunc.pdf


LLM response has improper format for chunk_index=13
LLM response has improper format for chunk_index=26
LLM response has improper format for chunk_index=25
LLM response has improper format for chunk_index=31
LLM response has improper format for chunk_index=35
LLM response has improper format for chunk_index=34
LLM response has improper format for chunk_index=47
LLM response has improper format for chunk_index=55
LLM response has improper format for chunk_index=68
LLM response has improper format for chunk_index=81


Result: run_id='fbccc397-43c4-4678-aa9b-ed2cf1b3200b' result={'resolver': {'number_of_nodes_to_resolve': 602, 'number_of_created_nodes': 484}}


# Query

In [21]:
from neo4j_graphrag.indexes import create_vector_index, drop_index_if_exists
from neo4j_graphrag.generation.prompts import RagTemplate
from neo4j_graphrag.generation.graphrag import GraphRAG

drop_index_if_exists(neo4j_driver, name="text_embeddings")
create_vector_index(neo4j_driver, name="text_embeddings", label="Chunk",
                   embedding_property="embedding", dimensions=768, similarity_fn="cosine")

# Vector Retriever
from neo4j_graphrag.retrievers import VectorRetriever

vector_retriever = VectorRetriever(
   neo4j_driver,
   index_name="text_embeddings",
   embedder=embedder,
   return_properties=["text"],
)

# GraphRAG Vector Cypher Retriever
from neo4j_graphrag.retrievers import VectorCypherRetriever

graph_retriever = VectorCypherRetriever(
    neo4j_driver,
    index_name="text_embeddings",
    embedder=embedder,
    retrieval_query="""
//1) Go out 2-3 hops in the entity graph and get relationships
WITH node AS chunk
MATCH (chunk)<-[:FROM_CHUNK]-(entity)-[relList:!FROM_CHUNK]-{1,2}(nb)
UNWIND relList AS rel

//2) collect relationships and text chunks
WITH collect(DISTINCT chunk) AS chunks, collect(DISTINCT rel) AS rels

//3) format and return context
RETURN apoc.text.join([c in chunks | c.text], '\n') +
  apoc.text.join([r in rels |
  startNode(r).name+' - '+type(r)+' '+r.details+' -> '+endNode(r).name],
  '\n') AS info
"""
)

rag_template = RagTemplate(template='''Answer the Question using the following Context. Only respond with information mentioned in the Context. Do not inject any speculative information not mentioned.

# Question:
{query_text}

# Context:
{context}

# Answer:
''', expected_inputs=['query_text', 'context'])

vector_rag  = GraphRAG(llm=llm, retriever=vector_retriever, prompt_template=rag_template)

graph_rag = GraphRAG(llm=llm, retriever=graph_retriever, prompt_template=rag_template)

q = "Can you summarize systemic lupus erythematosus (SLE)? including common effects, biomarkers, and treatments? Provide in detailed list format."

vector_rag.search(q, retriever_config={'top_k':5}).answer
graph_rag.search(q, retriever_config={'top_k':5}).answer

'Based on the provided information, here are some key points and answers related to lupus (Systemic Lupus Erythematosus or SLE):\n\n1. **Pathophysiology and Biomarkers:**\n   - Type-1 interferon gene expression is a biomarker for lupus.\n   - miR-146a is associated with type-1 interferon gene expression in lupus.\n   - The pathophysiologic pathway of lupus involves handling of apoptotic debris, chromatin, and immune complexes.\n\n2. **Treatment:**\n   - Hydroxychloroquine treats severe cutaneous lupus and lupus.\n   - Oral steroids treat severe cutaneous lupus and lupus.\n   - Rituximab is associated with renal lupus and treats lupus APS (Antiphospholipid Syndrome) arterial thrombolysis.\n\n3. **Assessment Tools:**\n   - SLEDAI scoring systems assess lupus, renal lupus, and cerebral lupus.\n   - BILAG assesses lupus.\n\n4. **Associations:**\n   - Lupus is associated with specific organ damage, non-coding RNAs, histone modifications, and innate immune activation and signaling.\n   - Cer